# Introduction

* **This is notebook for absolute beginner who is learning TensorFlow framework.**
* First go through the Traffic Sign Dataset which I used for this notebook. I will be highly obliged if you upvote the dataset also😊.
* Basically this is a Multi class classification using Transfer Learning in Tensorflow.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import tensorflow as tf
import pathlib
import PIL
import time
import zipfile
import random
from tensorflow.keras.layers import *

# Loading Data

In [ ]:
MAIN_PATH = "../input/traffic-sign-cropped/crop_dataset/crop_dataset"
TEST_PATH = "../input/traffic-sign-cropped/test_data"
CLASSES = os.listdir(MAIN_PATH)
NUM_CLASSES = len(CLASSES)
NUM_TEST_IMAGES = len(os.listdir(os.path.join(TEST_PATH,"test_data")))
HEIGHT,WIDTH = 32,32
BATCH_SIZE = 32
SPLIT = 0.2

# Preprocessing Data

### Go through the [Tensorflow preprocessing](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) image data documentation for more details

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    validation_split=SPLIT)
'''
    width_shift_range=0.2,
    height_shift_range=0.2,
    '''
train_ds = train_datagen.flow_from_directory(
    MAIN_PATH,
    target_size = (HEIGHT,WIDTH),
    batch_size = BATCH_SIZE,
    subset = "training",
    class_mode = "categorical",
    shuffle = True
)

val_ds = train_datagen.flow_from_directory(
    MAIN_PATH,
    target_size = (HEIGHT,WIDTH),
    batch_size = BATCH_SIZE,
    subset = "validation",
    class_mode = "categorical",
    shuffle = True
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)
test_ds = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size = (HEIGHT,WIDTH),
    shuffle = False
)

# Creating model

### I am using transfer learning for this task. Based on the number of elements of 'ls', I will be varying the number of pretrained model I am using. If I use more than one pretrained model, I will take average of all the outputs.

In [ ]:
def create_model():
    vgg16 = tf.keras.applications.VGG16(include_top=False, weights='imagenet',input_shape=[HEIGHT,WIDTH, 3])
            
    x = vgg16.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.3) (x)
    x = tf.keras.layers.Dense(128) (x)
    x = tf.keras.layers.LeakyReLU(alpha=0.2) (x)
    x = tf.keras.layers.GaussianDropout(0.4) (x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES,activation="softmax", dtype='float32')(x)
        
    model = tf.keras.Model(vgg16.input, outputs)
    return model

model = create_model()
model.summary()

## Function for compiling the model

In [ ]:
def compile_model(model, lr=0.0001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
        
    metrics = [
       tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

## Function for callbacks

In [ ]:
def create_callbacks():
    
    cpk_path = './best_model.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor='val_categorical_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1,
    )

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_categorical_accuracy',
        mode='max',
        factor=0.1,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor='val_categorical_accuracy',
        mode='max',
        patience=10, 
        verbose=1
    )
    
    callbacks = [checkpoint, reducelr, earlystop]         
    
    return callbacks

## Training

In [ ]:
EPOCHS= 60
VERBOSE =1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    model = create_model()
    model = compile_model(model, lr=0.0001)
   
    callbacks = create_callbacks()
    
    history = model.fit(train_ds, 
                        epochs=EPOCHS,
                        callbacks=callbacks,
                        validation_data = val_ds,
                        verbose=VERBOSE)

# METRICS VISUALIZATION

In [ ]:
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(history.history['val_loss']))
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Categorical Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Categorical Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Categorical Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# TESTING

In [ ]:
label_df = pd.read_csv("../input/traffic-sign-cropped/test_labels.csv")
labels = np.array(label_df.label)

model = tf.keras.models.load_model("./best_model.h5")

pred_arr = model.predict(test_ds)

predictions = np.zeros((NUM_TEST_IMAGES,))
for i,pred in enumerate(pred_arr):
    predictions[i] = np.argmax(pred)
    
confusion_matrix = tf.math.confusion_matrix(labels,predictions,num_classes=NUM_CLASSES)

count = 0
for i in range(NUM_TEST_IMAGES):
    if predictions[i]== labels[i]:
        count+=1
        
test_acc = count/NUM_TEST_IMAGES
print("Test Accuracy: ",test_acc)

In [ ]:
import seaborn as sns

sns.set(rc={'figure.figsize':(13,10)})

sns.heatmap(confusion_matrix,cmap='Blues')  

### Hope you learnt something useful from this notebook. Feel free to contact me in case of doubts or any suggestions. 
## Happy coding❤